In [13]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
    text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
    + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [75]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text = line[:-3]
            label = line[-2]
            # label2= int(label)
            
            
    
            yield text, label
    f.close()

In [76]:
next(stream_docs(path='movie_data.csv'))

('election is a chinese mob movie or triads in this case every two years an election is held to decide on a new leader and at first it seems a toss up between big d tony leung ka fai or as i know him the other tony leung and lok simon yam who was judge in full contact though once lok wins big d refuses to accept the choice and goes to whatever lengths he can to secure recognition as the new leader unlike any other asian film i watch featuring gangsters this one is not an action movie it has its bloody moments when necessary as in goodfellas but it s basically just a really effective drama there are a lot of characters which is really hard to keep track of but i think that plays into the craziness of it all a bit a 100 year old baton which is the symbol of power i mentioned before changes hands several times before things settle down and though it may appear that the film ends at the 65 or 70 minute mark there are still a couple big surprises waiting simon yam was my favorite character 

In [77]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
            return None, None
    return docs, y

In [78]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', n_features=2**21, preprocessor=None, tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1)#, n_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [80]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

ValueError: classes should include all valid labels that can be in y

In [55]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

AttributeError: 'SGDClassifier' object has no attribute 'coef_'

In [ ]:
clf = clf.partial_fit(X_test, y_test)

In [12]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest): os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'),'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

NameError: name 'clf' is not defined

# LDA with scikit-learn

In [56]:
import pandas as pd
df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [57]:
count = CountVectorizer(stop_words='english',
    max_df=.1,
    max_features=5000)
X = count.fit_transform(df['review'].values)

NameError: name 'CountVectorizer' is not defined

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics=10,
    random_state=123,
    learning_method='batch')
X_topics = lda.fit_transform(X)

In [ ]:
lda.components_.shape

In [ ]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
    for i in topic.argsort()\
    [:-n_top_words - 1:-1]]))


In [ ]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')

# Serializing fitted scikit-learn estimators

In [ ]:
>>> import pickle
>>> import os
>>> dest = os.path.join('movieclassifier', 'pkl_objects')
>>> if not os.path.exists(dest):
... os.makedirs(dest)
>>> pickle.dump(stop,
... open(os.path.join(dest, 'stopwords.pkl'),'wb'),
... protocol=4)
>>> pickle.dump(clf,
... open(os.path.join(dest, 'classifier.pkl'), 'wb'),
... protocol=4)

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle
cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
os.path.join(cur_dir,
'pkl_objects',
'stopwords.pkl'), 'rb'))
def tokenizer(text):
text = re.sub('<[^>]*>', '', text)
emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
text.lower())
text = re.sub('[\W]+', ' ', text.lower()) \
+ ' '.join(emoticons).replace('-', '')
tokenized = [w for w in text.split() if w not in stop]
return tokenized
vect = HashingVectorizer(decode_error='ignore',
n_features=2**21,
preprocessor=None,
tokenizer=tokenizer)

In [ ]:
>>> import pickle
>>> import re
>>> import os
>>> from vectorizer import vect
>>> clf = pickle.load(open(
... os.path.join('pkl_objects',
... 'classifier.pkl'), 'rb'))

In [ ]:
>>> import numpy as np
>>> label = {0:'negative', 1:'positive'}
>>> example = ['I love this movie']
>>> X = vect.transform(example)
>>> print('Prediction: %s\nProbability: %.2f%%' %\
... (label[clf.predict(X)[0]],
... np.max(clf.predict_proba(X))*100))
Prediction: positive
Probability: 91.56%

# Setting up an SQLite database for data storage